In [ ]:
! pip install python-dotenv
! pip install transformers torch huggingface_hub > /dev/null
! pip install torch torchvision > /dev/null
! pip install transformers datasets 
! pip install accelerate>=0.26.0

In [ ]:
import os 
from dotenv import load_dotenv

TOKEN = os.getenv("TOKEN")
TOKENIZERS_PARALLELISM = os.getenv("TOKENIZERS_PARALLELISM")
from huggingface_hub import login
login(token = TOKEN)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it")

In [3]:
from datasets import load_dataset 
dataset = load_dataset('csv', data_files = "fol_expressions_20000.csv")['train']
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_set = split_dataset['train']
test_set = split_dataset['test']

In [4]:
print(dataset.column_names)

['expression', 'english_description']


In [5]:
def preprocess(batch):
     inputs = tokenizer(
         batch['expression'],
         max_length=128,
         truncation=True,
         padding = 'max_length',
         
     )    
    
     labels = tokenizer(
         batch['english_description'],
         max_length =128,
         truncation = True, 
         padding = 'max_length'
     )
     inputs['labels']=labels['input_ids']
     return inputs

In [6]:
train_tokenized = train_set.map(preprocess, batched=True)
test_tokenized = test_set.map(preprocess, batched=True)

Map: 100%|██████████| 4000/4000 [00:00<00:00, 8990.79 examples/s]


In [9]:
#setting the trainer 
from transformers import Trainer, TrainingArguments 


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
)

trainer.train()

Step,Training Loss


KeyboardInterrupt: 